In [ ]:
import telegram
from telegram.ext import Updater
from telegram.ext import CommandHandler
from telegram.ext import MessageHandler, Filters
import requests 
import json
import pandas as pd
from bs4 import BeautifulSoup
import logging
import emoji
import schedule
import time
import threading

# JSON data read
with open ('./stock_data.json',encoding = 'UTF-8') as json_file:
    json_data = json.load(json_file)
    json_data_2 = json_data['stock']['dataValues']

# make table using stock_data.json file
info_table = []
for idx,info in enumerate(json_data_2):
    code = info['stock_number']
    code_name = info['stock_name']
    price = info['stock_price']
    info_table.append([])
    info_table[idx].append(idx)
    info_table[idx].append(code)
    info_table[idx].append(price)
    
# Active Bot with the token
token = "1461066934:AAHQLhV3b7PJ-EJX-JNJdYv0tz2fZBnWmhs"
bot = telegram.Bot(token)

# updater 
updater = Updater(token=token, use_context=True)
dispatcher = updater.dispatcher

# Pre-requirement Data for send_stock_recommendation
url = "https://finance.naver.com/item/main.nhn?code=" 
ment1 = "<<  끝투's 오늘의 추천 종목  >>\n\n"
ment = ment1
image = 'test_image.jpg'

# user_list = ["1437875774","1452320827" ]
user_list = ["1437875774" ]

# help page
help_text1 = "<<  끝투 사용방법 안내 >>\n\n1.  /help : 도움말 페이지 \n\n"
help_text2 = "2.  /recommend : 오늘의 추천 주식 \n\n3.  /buy : 주식구매 명령어"
help_text3 = "\n                 ex) 3번 주식을 20개 매수하려면\n                 /buy 3 20 을 입력하세요 \n"
help_text = help_text1 + help_text2 + help_text3
greeting_text = "끝투에 오신 것을 환영합니다!!\n"

# Getting Stock Information
def get_stock_price(code):
    result = requests.get(url+code)
    bs_obj = BeautifulSoup(result.content,"html.parser")
    no_today = bs_obj.find("p", {"class": "no_today"})
    blind = no_today.find("span", {"class": "blind"}) # 태그 span, 속성값 blind 찾기
    now_price = blind.text
    return now_price

# make stock name as same length with each other
def equalizer(a):
    gong = ""
    for i in range (9-a):
        gong = gong + "  "
    return gong    

# make 'ment' for sending telegram
def send_stock_recommendation(json_data_2,user_id):
    now = time.localtime()
    url = "https://finance.naver.com/item/main.nhn?code=" 
    ment1 = "<<  끝투's 오늘의 추천 종목  >>\n"
    ment2 =  "현재시각 : %04d/%02d/%02d %02d:%02d:%02d\n\n" % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)
    ment = ment1 + ment2
    image = 'test_image.jpg'    
    for idx,info in enumerate(json_data_2):
        code = info['stock_number']
        code_name = info['stock_name']
        name_len_a = len(code_name)
        price = info['stock_price']
        price = str(price)
        price = price.rjust(8)
        url_with_code = url + code
        updated_url = "[" + code_name + "]" + "(" + url_with_code + ")"
        temp = str(idx+1) + '. ' + updated_url + equalizer(name_len_a) +'가격'+price + '    매수 클릭\n\n'   ####you can use get_stock_price(code) for the price
        ment = ment + temp
    # send message to users     
    bot.sendMessage(chat_id = user_id,text = ment, parse_mode = 'Markdown',disable_web_page_preview=True)

# Defalult Greeting to all members
for user_id in user_list:   
    bot.send_photo(chat_id  = user_id, photo = open(image, 'rb'))
    bot.sendMessage(chat_id = user_id, text  = greeting_text)
    bot.sendMessage(chat_id = user_id, text  = help_text)    

# help page    
def help(update, context):
    context.bot.send_message(chat_id=update.effective_chat.id, text= help_text)
    print(update.effective_chat.id)

start_handler = CommandHandler('help', help)
dispatcher.add_handler(start_handler)

# send stock_recommendation
def recommend(update, context):
    send_stock_recommendation(json_data_2,update.effective_chat.id)

start_handler = CommandHandler('recommend', recommend)
dispatcher.add_handler(start_handler)

#for priodical report
def report():
    print("aaa\n")
    for user_id in user_list:
        send_stock_recommendation(json_data_2,user_id)

# start
def start(update, context):
    context.bot.send_message(chat_id=update.effective_chat.id, text="I'm a bot, please talk to me!")

start_handler = CommandHandler('start', start)
dispatcher.add_handler(start_handler)

# sub-function for 'buy' function. 
def find_matching_stock_name(user_choice):
    return info_table[int(user_choice)-1][1]

# sub-function for 'buy' function. 
def find_matching_stock_price(user_choice):
    return info_table[int(user_choice)-1][2]

# start
def buy(update, context):
    user_want = update.message.text
    buy_commend,user_choice,user_quantity = user_want.split()
    user_stock_number = find_matching_stock_name(user_choice)
    user_stock_price = find_matching_stock_price(user_choice)
    buy_text1 = "고객님이 선택하신 ["
    buy_text2 = "번] 주식을 ["
    buy_text3 = "원]에 ["
    buy_text4 = "]개 매수 하였습니다."
    context.bot.send_message(chat_id=update.effective_chat.id, 
                             text=buy_text1+user_choice+buy_text2+user_stock_price+buy_text3+user_quantity+buy_text4)
    user_response_data = {
      "response": {
        "dataValues": [
            {
            "stock_number": user_stock_number,
            "stock_quantity" : user_quantity
            }
        ]
      }
    }
    with open("user_response.json", "w", encoding = "UTF-8") as json_file:
        json.dump(user_response_data, json_file, indent=4, sort_keys=True)

start_handler = CommandHandler('buy', buy)
dispatcher.add_handler(start_handler)

# def telegram_bot_sendtext(bot_message):
#     bot_token = '1461066934:AAHQLhV3b7PJ-EJX-JNJdYv0tz2fZBnWmhs'
#     bot_chatID = '1437875774'
#     send_text = 'https://api.telegram.org/bot' + bot_token + '/sendMessage?chat_id=' + bot_chatID + '&parse_mode=Markdown&text=' + bot_message
#     response = requests.get(send_text)
#     return response.json()

# def report():
#     my_balance = 10   ## Replace this number with an API call to fetch your account balance
#     my_message = "Current balance is: {}".format(my_balance)   ## Customize your message
#     telegram_bot_sendtext(my_message)


# # schedule.every(2).minutes.do(report)
# schedule.every(1).seconds.do(report)

# original version
# while True:
#     updater.start_polling(timeout=3, clean=True)
#     updater.idle()
#     schedule.run_pending()
#     time.sleep(1)
    
# while True:
#     updater.start_polling(timeout=3, clean=True)
#     updater.idle()
#     updater.stop()
#     schedule.run_pending()
#     time.sleep(1)

schedule.every(60).seconds.do(report)
def priodical_stock_data_report(interval):
    while True:
        schedule.run_pending()
        time.sleep(10) 

    
t = threading.Thread(target = priodical_stock_data_report, args=(30,))
t.start()
print("main-thread\n")

while True:
    updater.start_polling(timeout=3, clean=False)
    updater.idle()
    time.sleep(1)
    

main-thread

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

aaa

1437875774
aaa

aaa

